<a href="https://colab.research.google.com/github/RonSK03/RNN_LSTM/blob/main/RNN%2BLSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# IMPORTANT: SOME KAGGLE DATA SOURCES ARE PRIVATE
# RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES.
import kagglehub
kagglehub.login()


In [ ]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

rmisra_news_category_dataset_path = kagglehub.dataset_download('rmisra/news-category-dataset')
ronsk03_glove_dataset_path = kagglehub.dataset_download('ronsk03/glove-dataset')
hassanamin_glove_based_text_classification_path = kagglehub.notebook_output_download('hassanamin/glove-based-text-classification')

print('Data source import complete.')


In [ ]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("rmisra/news-category-dataset")

print("Path to dataset files:", path)

In [ ]:
!pip install --no-cache-dir torch==2.1.2 torchvision==0.16.2 torchaudio==2.1.2 torchtext==0.16.2 --index-url https://download.pytorch.org/whl/cu118


In [ ]:
import torch
import torchtext

# This should now work without any errors.
print(f"PyTorch Version: {torch.__version__}")
print(f"Torchtext Version: {torchtext.__version__}")


In [ ]:
import torch
device=torch.device('cpu')
if torch.cuda.is_available():
    device=torch.device('cuda')

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
data_full=pd.read_json('/kaggle/input/news-category-dataset/News_Category_Dataset_v3.json', lines=True)

In [ ]:
data_full.to_csv('news_dataset.csv', index=False)


In [ ]:
data_full

In [ ]:
features=data_full[['headline' ,'category' , 'short_description']]

In [ ]:
features.head(5)

In [ ]:
text=features[['headline', 'short_description']]

In [ ]:
text.head(5)

In [ ]:
text=pd.DataFrame({'headline':text['headline'].str.lower(),
                   'short_description':text['short_description'].str.lower()})

In [ ]:
text.head(5)

In [ ]:
import nltk

In [ ]:
nltk.download('all')

In [ ]:
from nltk import word_tokenize
from nltk.corpus import stopwords

In [ ]:
stop_words = set(stopwords.words('english'))


In [ ]:
text['headline']=text['headline'].apply(word_tokenize)
text['short_description']=text['short_description'].apply(word_tokenize)

In [ ]:
text

In [ ]:
def stop_removal(tokend):
  temp=[]
  for w in tokend:
    if w not in stop_words:
      temp.append(w)
  return temp

In [ ]:
text['headline']=text['headline'].apply(stop_removal)
text['short_description']=text['short_description'].apply(stop_removal)


In [ ]:
text

In [ ]:
punc=[' ', ';', ':', ',','&','(', ')', '.']
def punc_removal(tokend):
  temp=[]
  for w in tokend:
    if w not in punc:
      temp.append(w)
  return temp

In [ ]:
text['headline']=text['headline'].apply(punc_removal)
text['short_description']=text['short_description'].apply(punc_removal)


In [ ]:
text

In [ ]:
import torchtext
from torchtext.vocab import build_vocab_from_iterator

In [ ]:
import torchtext
from torchtext.vocab import build_vocab_from_iterator

all_tokens = (text['headline'] + text['short_description']).tolist()
specials = ['<UNK>', '<PAD>']
final_vocab = build_vocab_from_iterator(all_tokens, specials=specials, min_freq=5)

UNK_IDX = final_vocab['<UNK>']
final_vocab.set_default_index(UNK_IDX)



In [ ]:
final_vocab

In [ ]:
category=features['category']

In [ ]:
category

In [ ]:
category.str.lower()

In [ ]:
from sklearn.preprocessing import LabelEncoder

lwr_categories = features['category'].str.lower()

label_encoder = LabelEncoder()
numerical_labels = label_encoder.fit_transform(lwr_categories)

NUM_CLASSES = len(label_encoder.classes_)



In [ ]:
from torch.nn.utils.rnn import pad_sequence


In [ ]:
tensors = []
for index, row in text.iterrows():
    combined_tokens = row['headline'] + row['short_description']
    numerical_input = final_vocab(combined_tokens)
    tensors.append(torch.tensor(numerical_input))
final_text = pad_sequence(tensors, batch_first=True, padding_value=final_vocab['<PAD>'])


In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(final_text, numerical_labels, test_size=0.2)


In [ ]:

def new_embedding(filepath, word_index, embedding_dim):
    vocab_size = len(word_index)
    embedding_matrix = np.zeros((vocab_size, embedding_dim))

    with open(filepath, encoding="utf8") as f:
        for line in f:
            word, *vector = line.split()
            if word in word_index:
                idx = word_index[word]
                embedding_matrix[idx] = np.array(
                    vector, dtype=np.float32)[:embedding_dim]

    return torch.from_numpy(embedding_matrix)


In [ ]:
glove_file_path = '/kaggle/input/glove-dataset/glove.6B.100d.txt'


In [ ]:
initial_weights = new_embedding(
    glove_file_path ,
    final_vocab.get_stoi(),
    100
)


In [ ]:
import torch.optim as optim
import torch.nn as nn

class LSTM(nn.Module):
    def __init__(self, vocab_size, embedded_dim, hidden_dim, output_dim, padding_idx, initial_weights):
        super(LSTM, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embedded_dim, padding_idx=padding_idx)
        self.embedding.weight.data.copy_(initial_weights)
        self.embedding.weight.requires_grad = False

        self.hidden_dim = hidden_dim
        self.forget=nn.Linear(embedded_dim+hidden_dim, hidden_dim)
        self.input=nn.Linear(embedded_dim+hidden_dim, hidden_dim)
        self.output=nn.Linear(embedded_dim+hidden_dim, hidden_dim)
        self.candidate=nn.Linear(embedded_dim+hidden_dim, hidden_dim)
        self.fc = nn.Linear(hidden_dim, output_dim)
        self.dropout = nn.Dropout(0.3)

    def forward(self, text):
        embedded=self.embedding(text)
        batch_size, no_tokens, _ = embedded.shape
        h_t_1 = torch.zeros(batch_size, self.hidden_dim).to(device)
        c_t_1 = torch.zeros(batch_size, self.hidden_dim).to(device)
        for t in range(no_tokens):
            x_t = embedded[:, t, :]
            f_t = torch.cat((x_t, h_t_1), dim=1)
            f_t=self.forget(f_t)
            f_t=torch.sigmoid(f_t)
            c1_t=self.candidate(torch.cat((x_t , h_t_1), dim=1))
            c1_t=torch.tanh(c1_t)
            i_t=self.input(torch.cat((x_t , h_t_1), dim=1))
            i_t=torch.sigmoid(i_t)
            c_t = f_t * c_t_1 + i_t * c1_t
            o_t=self.output(torch.cat((x_t , h_t_1), dim=1))
            o_t=torch.sigmoid(o_t)
            h_t=o_t*(torch.tanh(c_t))
            h_t_1 = h_t
            c_t_1 = c_t
        h_t=self.dropout(h_t)


        return self.fc(h_t)

In [ ]:
device=torch.device('cpu')
if torch.cuda.is_available():
    device=torch.device('cuda')
    print("cuda using")

In [ ]:
from torch.utils.data import Dataset, DataLoader

class Datasetmaking(Dataset):
    def __init__(self, X, y):
        if isinstance(X, torch.Tensor):
            self.X = X
        else:
            self.X = torch.tensor(list(X), dtype=torch.long)

        if isinstance(y, torch.Tensor):
            self.y = y
        else:
            self.y = torch.tensor(list(y), dtype=torch.long)

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]


train_datas = Datasetmaking(X_train, y_train)
trainer = DataLoader(train_datas, batch_size=32, shuffle=True)


In [ ]:
test_datas=Datasetmaking(X_test, y_test)
tester=DataLoader(test_datas, batch_size=32, shuffle=False)

In [ ]:
import torch.optim as optim

model = pyRNN(
    vocab_size=len(final_vocab),
    embedding_dim=100,
    hidden_dim=256,
    output_dim=NUM_CLASSES,
    n_layers=2,
    dropout=0.5,
    pad_idx=final_vocab['<PAD>']
).to(device)

model.embedding.weight.data.copy_(initial_weights)
model.embedding.weight.requires_grad = False

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-3)

for epoch in range(10):
    model.train()
    for batch_X, batch_y in trainer:
        batch_X, batch_y = batch_X.to(device), batch_y.to(device)

        optimizer.zero_grad()
        output = model(batch_X)
        loss = criterion(output, batch_y)
        loss.backward()
        optimizer.step()

    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for batch_X, batch_y in tester:
            batch_X, batch_y = batch_X.to(device), batch_y.to(device)
            outputs = model(batch_X)
            predictions = torch.argmax(outputs, dim=1)
            correct += (predictions == batch_y).sum().item()
            total += batch_y.size(0)

    accuracy = correct / total * 100
    print(f"Epoch {epoch+1}: Test Accuracy = {accuracy:.2f}%")

In [ ]:
import torch.nn as nn

class pyLSTM(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, output_dim, n_layers,
                 dropout, pad_idx):

        super().__init__()

        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx=pad_idx)

        self.lstm = nn.LSTM(embedding_dim,
                           hidden_dim,
                           num_layers=n_layers,
                           batch_first=True)

        self.fc = nn.Linear(hidden_dim, output_dim)

        self.dropout = nn.Dropout(dropout)

    def forward(self, text):
        embedded = self.embedding(text)

        outputs, (hidden, cell) = self.lstm(embedded)

        hidden = self.dropout(hidden[-1,:,:])

        return self.fc(hidden)

In [ ]:
import torch.optim as optim

model = pyLSTM(
    vocab_size=len(final_vocab),
    embedding_dim=100,
    hidden_dim=256,
    output_dim=NUM_CLASSES,
    n_layers=2,
    dropout=0.5,
    pad_idx=final_vocab['<PAD>']
).to(device)

model.embedding.weight.data.copy_(initial_weights)
model.embedding.weight.requires_grad = False

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-3)

for epoch in range(10):
    model.train()
    for batch_X, batch_y in trainer:
        batch_X, batch_y = batch_X.to(device), batch_y.to(device)

        optimizer.zero_grad()
        output = model(batch_X)
        loss = criterion(output, batch_y)
        loss.backward()
        optimizer.step()

    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for batch_X, batch_y in tester:
            batch_X, batch_y = batch_X.to(device), batch_y.to(device)
            outputs = model(batch_X)
            predictions = torch.argmax(outputs, dim=1)
            correct += (predictions == batch_y).sum().item()
            total += batch_y.size(0)

    accuracy = correct / total * 100
    print(f"Epoch {epoch+1}: Test Accuracy = {accuracy:.2f}%")

In [ ]:
import torch.optim as optim

model = pyRNN(
    vocab_size=len(final_vocab),
    embedding_dim=100,
    hidden_dim=256,
    output_dim=NUM_CLASSES,
    n_layers=2,
    dropout=0.5,
    pad_idx=final_vocab['<PAD>']
).to(device)

model.embedding.weight.data.copy_(initial_weights)
model.embedding.weight.requires_grad = False

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-3)

for epoch in range(10):
    model.train()
    for batch_X, batch_y in trainer:
        batch_X, batch_y = batch_X.to(device), batch_y.to(device)

        optimizer.zero_grad()
        output = model(batch_X)
        loss = criterion(output, batch_y)
        loss.backward()
        optimizer.step()

    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for batch_X, batch_y in tester:
            batch_X, batch_y = batch_X.to(device), batch_y.to(device)
            outputs = model(batch_X)
            predictions = torch.argmax(outputs, dim=1)
            correct += (predictions == batch_y).sum().item()
            total += batch_y.size(0)

    accuracy = correct / total * 100
    print(f"Epoch {epoch+1}: Test Accuracy = {accuracy:.2f}%")

In [ ]:
import torch.nn as nn

class RNN(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, output_dim, padding_idx, initial_weights):
        super(RNN, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx=padding_idx)
        self.embedding.weight.data.copy_(initial_weights)
        self.embedding.weight.requires_grad = False

        self.i2h = nn.Linear(embedding_dim, hidden_dim)
        self.h2h = nn.Linear(hidden_dim, hidden_dim)
        self.dropout = nn.Dropout(0.3)
        self.fc = nn.Linear(hidden_dim, output_dim)

    def forward(self, text):
        embedded = self.embedding(text)
        batch_size, no_tokens, _ = embedded.shape
        h_t = torch.zeros(batch_size, self.h2h.out_features).to(device)
        h_t_1=torch.zeros(batch_size, self.h2h.out_features).to(device)

        for t in range(no_tokens):
            x_t = embedded[:, t, :]  # [batch_size, embedding_dim]
            h_t = torch.tanh(self.i2h(x_t) + self.h2h(h_t_1))
            h_t_1=h_t
        out = self.dropout(h_t)
        out=self.fc(out)
        return out


In [ ]:
import torch.optim as optim

model = LSTM(
    vocab_size=len(final_vocab),
    embedded_dim=100,
    hidden_dim=128,
    output_dim=42,
    padding_idx=final_vocab['<PAD>'],
    initial_weights=initial_weights
).to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-3)

for epoch in range(10):
    model.train()
    for batch_X, batch_y in trainer:
        batch_X, batch_y = batch_X.to(device), batch_y.to(device)

        optimizer.zero_grad()
        output = model(batch_X)
        loss = criterion(output, batch_y)
        loss.backward()
        optimizer.step()

    model.eval()
    correct = 0
    total = 0

    with torch.no_grad():
        for batch_X, batch_y in tester:
            batch_X, batch_y = batch_X.to(device), batch_y.to(device)
            outputs = model(batch_X)
            predictions = torch.argmax(outputs, dim=1)
            correct += (predictions == batch_y).sum().item()
            total += batch_y.size(0)

    accuracy = correct / total * 100
    print(f"Epoch {epoch+1}: Test Accuracy = {accuracy:.2f}%")
